In [41]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import os
import xml
import read_txt_class as rtc

## def class_cal(label_path) - 클래스별 value_counts 결과값
## label_path : txt폴더가 있는 디렉토리

## def make_key(basic_path) - 인덱스를 만들어줌, 채널+날짜+시간
## label_path : txt폴더가 있는 디렉토리

In [9]:
os.getcwd()

'd:\\Lee_highway'

In [11]:
os.listdir('/Lee_highway/bbox/train/box/image')

['Suwon_CH01_20200720_1700_MON_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200720_1730_MON_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200720_1800_MON_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200720_1900_MON_9m_RH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200720_1930_MON_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200720_2000_MON_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200720_2030_MON_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1530_TUE_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1600_TUE_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1630_TUE_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1730_TUE_9m_RH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1800_TUE_9m_RH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1900_TUE_9m_RH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_1930_TUE_9m_RH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_2030_TUE_9m_NH_highway_TW5_sunny_FHD',
 'Suwon_CH01_20200721_21

In [12]:
# 목표 : 수도권 영동선을 지나는 차량의 종류를 구분하는 모델 성능 향상
# 수단 : 데이터와 하이퍼파라미터 조정
# 딥러닝 모델 : yolov8
# 데이터 출처 : AIHUB 고속도로 데이터 

In [14]:
# 파일 데이터 활용해서 데이터 프레임 만들기

# D:\Lee_highway\bbox\train\box
home_path = '/Lee_highway/bbox/train/box'
data_path = os.listdir(f'{home_path}/image')
datas = []
column = ['지역', '채널', '날짜', '시간', '요일', '카메라높이', '정체여부', '도로구분', '차선정보', '날씨', '화질']

for i in range(len(data_path)):
    datas.append(data_path[i].split('_'))

high_df = pd.DataFrame(data=datas, columns=column)
high_df.head(3)

,지역,채널,날짜,시간,요일,카메라높이,정체여부,도로구분,차선정보,날씨,화질
0,Suwon,CH01,20200720,1700,MON,9m,NH,highway,TW5,sunny,FHD
1,Suwon,CH01,20200720,1730,MON,9m,NH,highway,TW5,sunny,FHD
2,Suwon,CH01,20200720,1800,MON,9m,NH,highway,TW5,sunny,FHD


In [15]:
# 필요 없는 컬럼 삭제
high_df = high_df.drop(['도로구분', '차선정보','화질'], axis=1)
high_df.head()

,지역,채널,날짜,시간,요일,카메라높이,정체여부,날씨
0,Suwon,CH01,20200720,1700,MON,9m,NH,sunny
1,Suwon,CH01,20200720,1730,MON,9m,NH,sunny
2,Suwon,CH01,20200720,1800,MON,9m,NH,sunny
3,Suwon,CH01,20200720,1900,MON,9m,RH,sunny
4,Suwon,CH01,20200720,1930,MON,9m,NH,sunny


In [16]:
# 컬럼 내용 정리 
# 지역 : CCTV 촬영 지역
# 채널 : CCTV 종류, 채널 별로 차량의 앞 혹은 뒤 이미지 구분
# 날짜 : 년월일 날짜
# 시간 : 24시간 기준, 낮과 저녁 구분
# 요일 
# 카메라가 설치된 높이
# 날씨
# 화질 : 이미지 화질

In [17]:
# 컬럼별 고유값 확인 
for i in high_df.columns:
    print(high_df[i].unique())

['Suwon']
['CH01' 'CH02' 'CH03' 'CH04']
['20200720' '20200721' '20200722' '20201010' '20201011' '20201012'
 '20201213']
['1700' '1730' '1800' '1900' '1930' '2000' '2030' '1500' '1530' '1600'
 '1630' '2100' '1330' '1400' '1830' '1653' '1753' '1823' '1853' '1923'
 '0700' '0730' '0930' '1030' '2130' '1430' '1807' '1837' '1908' '1736'
 '1836' '1906' '1657' '1728' '1828' '1858' '1928' '0732' '1033' '1204'
 '1742' '1812' '1842' '1913' '1843' '1943' '1732' '1802' '1833' '1903'
 '0704' '0835' '0936' '1137' '1717' '1748' '1849' '1818' '1919' '1706'
 '1737' '1939' '0737' '0838' '1142']
['MON' 'TUE' 'WED' 'SAT' 'SUN']
['9m']
['NH' 'RH']
['sunny' 'rainy' 'snow']


In [18]:
high_df.head(1)

,지역,채널,날짜,시간,요일,카메라높이,정체여부,날씨
0,Suwon,CH01,20200720,1700,MON,9m,NH,sunny


In [19]:
## 연, 월, 일로 나누기
# high_df['날짜'] = pd.to_datetime(high_df['날짜'], format='%Y%m%d')

# high_df['연도'] = high_df['날짜'].dt.year
# high_df['월'] = high_df['날짜'].dt.month
# high_df['일'] = high_df['날짜'].dt.day

# high_df.head(3)

In [20]:
# high_df['시'] = high_df['시간'].str.slice(0,2)
# high_df['분'] = high_df['시간'].str.slice(2,4)
# high_df.head(3)

In [21]:
for i in high_df.columns:
    print(high_df[i].value_counts())

지역
Suwon    168
Name: count, dtype: int64
채널
CH01    43
CH02    42
CH03    42
CH04    41
Name: count, dtype: int64
날짜
20200721    39
20200722    39
20200720    29
20201012    19
20201010    14
20201011    14
20201213    14
Name: count, dtype: int64
시간
1800    12
1900    12
1930    10
1830     9
1730     9
        ..
1858     1
1928     1
0732     1
1033     1
1142     1
Name: count, Length: 65, dtype: int64
요일
MON    48
TUE    39
WED    39
SUN    28
SAT    14
Name: count, dtype: int64
카메라높이
9m    168
Name: count, dtype: int64
정체여부
NH    101
RH     67
Name: count, dtype: int64
날씨
sunny    136
rainy     18
snow      14
Name: count, dtype: int64


In [22]:
# type(int(high_df.loc[0, '시간'])) int 타입으로 바꾸기
high_df['시간'] = high_df['시간'].astype(int)
print(type(high_df.loc[0, '시간']))
print(high_df['시간'][0])

<class 'numpy.int32'>
1700


In [23]:
# 시간대별로 낮과 밤으로 나누기
def day_or_night(time):
    if time <= 1800 :
        return '낮'
    else :
        return '밤'

print(high_df['시간'][0])
day_or_night(high_df['시간'][0])

1700


'낮'

In [24]:
high_df['낮/밤'] = high_df['시간'].apply(day_or_night)
high_df.head(3)

,지역,채널,날짜,시간,요일,카메라높이,정체여부,날씨,낮/밤
0,Suwon,CH01,20200720,1700,MON,9m,NH,sunny,낮
1,Suwon,CH01,20200720,1730,MON,9m,NH,sunny,낮
2,Suwon,CH01,20200720,1800,MON,9m,NH,sunny,낮


In [25]:
# type(high_df['채널'][0])

# channel_num = 70

# if high_df['채널'][channel_num] == 'CH02' or high_df['채널'][channel_num] == 'CH04' :
#     print('앞')
# else :
#     print('뒤')

In [26]:
# ch01 뒤, ch02 앞, ch03 뒤, ch04 앞
# 차량 이미지 카테고라이징
def front_or_back(channel):
    if channel == 'CH01'or channel == 'CH03' :
        return '뒤'
    else :
        return '앞'

print(high_df['채널'][30])
print(front_or_back(high_df['채널'][0]))

CH01
뒤


In [27]:
high_df['앞/뒤'] = high_df['채널'].apply(front_or_back)
high_df.head(3)

,지역,채널,날짜,시간,요일,카메라높이,정체여부,날씨,낮/밤,앞/뒤
0,Suwon,CH01,20200720,1700,MON,9m,NH,sunny,낮,뒤
1,Suwon,CH01,20200720,1730,MON,9m,NH,sunny,낮,뒤
2,Suwon,CH01,20200720,1800,MON,9m,NH,sunny,낮,뒤


In [28]:
print(high_df['채널'].value_counts())
print(high_df['낮/밤'].value_counts())
print(high_df['앞/뒤'].value_counts())

채널
CH01    43
CH02    42
CH03    42
CH04    41
Name: count, dtype: int64
낮/밤
낮    94
밤    74
Name: count, dtype: int64
앞/뒤
뒤    85
앞    83
Name: count, dtype: int64


In [29]:
high_df.head(3)

,지역,채널,날짜,시간,요일,카메라높이,정체여부,날씨,낮/밤,앞/뒤
0,Suwon,CH01,20200720,1700,MON,9m,NH,sunny,낮,뒤
1,Suwon,CH01,20200720,1730,MON,9m,NH,sunny,낮,뒤
2,Suwon,CH01,20200720,1800,MON,9m,NH,sunny,낮,뒤


In [33]:
high_df['key'] = high_df['채널'] +'-'+ high_df['날짜'] +'-'+ high_df['시간'].astype(str)
high_df.head(3)

,지역,채널,날짜,시간,요일,카메라높이,정체여부,날씨,낮/밤,앞/뒤,key
0,Suwon,CH01,20200720,1700,MON,9m,NH,sunny,낮,뒤,CH01-20200720-1700
1,Suwon,CH01,20200720,1730,MON,9m,NH,sunny,낮,뒤,CH01-20200720-1730
2,Suwon,CH01,20200720,1800,MON,9m,NH,sunny,낮,뒤,CH01-20200720-1800


In [30]:
# import read_txt_class as rtc

## def class_cal(label_path) - 클래스별 value_counts 결과값
## label_path : txt폴더가 있는 디렉토리

## def make_key(basic_path) - 인덱스를 만들어줌, 채널+날짜+시간
## label_path : txt폴더가 있는 디렉토리

label_path = 'D:/Lee_highway/bbox/train/box/label/'
test_path = os.listdir(label_path)
datas = []
colomns = ['car', 'truck', 'bus']

keys = rtc.make_key(label_path)

for i in test_path :
    txt_path = label_path + i
    test = rtc.class_cal(txt_path)
    datas.append(test)

df = pd.DataFrame(data=datas, columns=colomns, index=keys)
df.head()

,car,truck,bus
CH01-20200720-1700,2036,1166,75
CH01-20200720-1730,2016,872,98
CH01-20200720-1800,2134,857,152
CH01-20200720-1900,1901,1101,138
CH01-20200720-1930,1449,771,130


In [39]:
df['key'] = df.index
df.head(3)

,car,truck,bus,key
CH01-20200720-1700,2036,1166,75,CH01-20200720-1700
CH01-20200720-1730,2016,872,98,CH01-20200720-1730
CH01-20200720-1800,2134,857,152,CH01-20200720-1800


In [40]:
merge_df = pd.merge(left=high_df, right=df, on ='key')
merge_df.head(3)

,지역,채널,날짜,시간,요일,카메라높이,정체여부,날씨,낮/밤,앞/뒤,key,car,truck,bus
0,Suwon,CH01,20200720,1700,MON,9m,NH,sunny,낮,뒤,CH01-20200720-1700,2036,1166,75
1,Suwon,CH01,20200720,1730,MON,9m,NH,sunny,낮,뒤,CH01-20200720-1730,2016,872,98
2,Suwon,CH01,20200720,1800,MON,9m,NH,sunny,낮,뒤,CH01-20200720-1800,2134,857,152


In [46]:
# csv 파일 저장하기
filepath = 'D:/Lee_highway/merge_df.csv'
merge_df.to_csv(filepath, encoding='utf-8')

In [49]:
pd.pivot_table(data = merge_df, values=['car', 'truck', 'bus'], index=['채널','낮/밤'],aggfunc='sum' )

bus    car  truck
채널   낮/밤                    
CH01 낮    2104  67630  31992
     밤    2150  37290  10757
CH02 낮    1196  41544  13957
     밤    1663  38760   8151
CH03 낮    1083  30849  11047
     밤    1582  30197   5534
CH04 낮    1049  29475   7357
     밤     886  19557   3479